In [28]:
import plotly.express as px
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go

In [76]:
# Main Timeline plot -------------------------------------------------------------------------------------------
alo_data = pd.read_csv('DataViz_2024/treated_data/ALO_data_tableau.csv')

alo_data['start.season'] = pd.to_datetime(alo_data['start.season'].astype(str), format='%Y')
alo_data['End.season'] = pd.to_datetime(alo_data['End.season'].astype(str), format='%Y')

color_map = {
    "Minardi": "black", 
    "Renault": "#fcd205", # renault yellow
    "McLaren": "#E56717", # papaya orange
    "Ferrari": "#F70D1A", # Ferrari red
    "Alpine": "#2673E2", # Alpine blue
    "Aston Martin": "#004225" # british racing green
}

text_color_map = {
    "Minardi": "gold",
    "Renault": "grey",
    "McLaren": "black",
    "Ferrari": "yellow",
    "Alpine": "white",
    "Aston Martin": "white"
}

# Assuming alo_data is already loaded and processed
current_year = datetime.now().year

# Create the Gantt chart with Plotly Express
fig = px.timeline(
    alo_data,
    x_start="start.season",
    x_end="End.season",
    y="Constructors",
    color="Constructors",
    color_discrete_map=color_map
)

# Convert Plotly Express figure to Plotly Graph Objects figure for custom adjustments
fig = go.Figure(fig)

# Adjusting the opacity for each trace based on the 'End.season' year
for trace in fig.data:
    end_years = alo_data[alo_data['Constructors'] == trace.name]['End.season'].dt.year
    opacities = [0.66 if year > 2024 else 1 for year in end_years]
    trace.marker.opacity = opacities

last_periods = alo_data.groupby('Constructors')['End.season'].max()

# Customize text color to contrast with the bar color and add annotations
for index, row in alo_data.iterrows():
    if row['End.season'] == last_periods[row['Constructors']]:
        # Calculate position for the annotation
        text_position = row['End.season'] - ((row['End.season'] - row['start.season']) / 10)
        fig.add_annotation(
            x=text_position, y=row['Constructors'],
            text=row['Constructors'],
            showarrow=False,
            font=dict(
                color=text_color_map[row['Constructors']],
                size=12,  # Adjust size as needed
                family='Calibri',  # You can specify the font family
                weight='bold'  # Making text bold
            ),
            xanchor='right',
            align='right')

# Reverse the y-axis
fig.update_yaxes(autorange="reversed")

# Hide the legend as it's redundant now
fig.update_layout(showlegend=False)

# Adding red lines for specific years
years_of_interest = ["2005-01-01", "2006-01-02"]
for year in years_of_interest:
    fig.add_shape(
        type="line",
        x0=year, x1=year,
        y0=-0.5, y1=len(alo_data['Constructors'].unique()) - 0.5,
        line=dict(color="red", width=1.5)
    )

# Adjust x-axis ticks to show every year and set an upper limit for the x-axis
fig.update_xaxes(
    tickmode='linear',
    dtick="M12",
    range=["2000-12-01", "2026-02-01"],
    showgrid=True,
    gridwidth=1,
    gridcolor='lightgrey'
)
fig.update_layout(
    title='Timeline of teams where Alonso drove',  # Adding a title
    title_x=0.5,  # Center the title
    paper_bgcolor='white',  # Set the background color around the plot
    plot_bgcolor='white'  # Set the background color of the plot itself
)
fig.show()

In [ ]:
# 2005 season ---------------------------------------------------------------------------------------------------
# The race for the title
season_2005 = pd.read_csv('DataViz_2024/treated_data/season_2005.csv')